# Introduzione

## Trattazione teorica

### Bspline

Questo notebook è una presentazione delle possibili applicazioni della libreria `pyBspline`, contenuta nel file omonimo.
Questa libreria contiene 3 classi, di cui 2 *di appoggio*, `shape` e `knot_vector`, che servono per definire i parametri di input per allocare un oggetto della classe `Bspline`, la terza e principale classe della libreria. 
Quest'ultima classe permette di gestire delle funzioni Bspline:
- scegliendo il grado polinomiale `P` delle funzioni di base dello spazio delle Bspline
- scegliendo il numero `N` di funzioni di base dello spazio
- scegliendo il knot vector `v`
- scegliendo la dimensionalità del dominio delle funzioni: è possibile usare curve Bspline, defininendo un solo knot vector, superfici Bspline definendo 2 knot vector, ecc...
- scegliendo la dimensionalità del codominio: se i control points della Bspline sono scalari allora si definisce una funzione scalare, se hanno valori vettoriali si descrivono curve e superfici.

### Equazione di Helmholtz

La libreria `pyBspline` è stata implementata per poter risolvere anche l'equazione di Helmholtz in $\mathbf{R}^2$ in domini non semplicemente connessi tramite l'approccio detto BEM: Boundary Element Method.

Data un'onda incidente $u^{inc}$ (soluzione dell'equazion di Helmholtz) su un ostacolo $\Omega_{-}$ con bordo $\Gamma$, vogliamo trovare la funzione $u^{scat}$ "emessa" dell'ostacolo affinchè siano valide le seguenti relazioni
\begin{align*}
\Delta u^{scat} + k^2 u^{scat} = 0 & \quad\text{in} \quad \quad\Omega_{+}\\
\gamma^{+} \left( u^{scat} + u^{inc} \right) = 0 & \quad\text{in} \quad\Gamma\\
u^{scat} \quad\text{è uscente}
\end{align*}

Questo è quello che viene detto *Sound-soft scattering problem*, o SSSP, a seguito delle condizioni al bordo $\Gamma$ imposte. Questo problema si può mostrare essere riconducibile al problema detto EDP (*Exterior Dirichlet problem*):
\begin{align*}
\Delta u + k^2 u = 0 & \quad\text{in} \quad \quad\Omega_{+}\\
\gamma^{+} u  = g_D & \quad\text{in} \quad\Gamma\\
u \quad\text{è uscente}
\end{align*}
SSSp è riconducibile a EDP se
\begin{align*}
u = u^{scat}\\
g_D = -  \gamma^{+} u^{inc}
\end{align*}

### BEM

Definendo la soluzione fondamentale dell'equazione di Helmholtz in $\mathbf{R}^2$
\begin{equation*}
\Phi_k \left(\mathbf{x},\mathbf{y}\right) \equiv \frac{i}{4} H^{\left(1\right)}_{0}\left(k\lvert \mathbf{x}-\mathbf{y}\rvert\right) \quad\text{con}\quad\mathbf{x}\neq \mathbf{y} \in \mathbf{R}^2
\end{equation*}
è pissibile prendere una qualunque sovrapposizione lineare di soluzioni fondamentali per generare un'altra soluzione dell'equazione di Helmholtz. 

Per trovare la corretta sovrapposizione lineare che porti alle condizioni al bordo richieste è sufficiente trovare $\psi$ definita in $\Gamma$ che risolva la BIE (*Boundary Integral Equation*) 
\begin{equation*}
S\psi = g_D \quad\text{in}\quad\Gamma
\end{equation*}
dove abbiamo definto il *single-layer operator* $S$ come
\begin{equation*}
\left(S\psi\right)\left( \mathbf{x}\right) = \int_{\Gamma} \Phi_k \left(\mathbf{x},\mathbf{y}\right) \psi \left(\mathbf{y}\right)ds\left(\mathbf{y}\right) \quad\text{con}\quad\mathbf{x} \in \Gamma
\end{equation*}

In questo modo è possibile trovare la soluzione $u$ del EDP tramite la *representation formula* 
\begin{equation*}
u = \mathcal{S}\psi \quad\text{in}\quad\Omega_{+}
\end{equation*}

dove abbiamo definito il *single-layer potential* $\mathcal{S}$ come
\begin{equation*}
\left(\mathcal{S}\psi\right)\left( \mathbf{x}\right) = \int_{\Gamma} \Phi_k \left(\mathbf{x},\mathbf{y}\right) \psi \left(\mathbf{y}\right)ds\left(\mathbf{y}\right) \quad\text{con}\quad\mathbf{x}\in \Omega_{+}
\end{equation*}

Notiamo subito che la differenza tra $S$ e $\mathcal{S}$ sta solo nel fatto che nel primo caso $\mathbf{x}\in\Gamma$ mentre nel socondo $\mathbf{x}\in\Omega_{+}$, di conseguenza si può mostrare che 
\begin{equation*}
S\psi = \gamma^{+}\left( \mathcal{S}\psi\right)
\end{equation*}



### Metodo di Galerkin

Per risolvere la BIE è possibile sfruttare il metodo di Galerkin, cioè restringere lo spazio delle funzioni definite in $\Gamma$ ad uno spazio discreto $V_N$ e trovare $\psi_N\in V_N$ tale per cui
\begin{equation*}
\mathcal{A}\left(\psi_N,\xi_N\right) = \int_{\Gamma} \left( S \psi_N \right) \bar{\xi}_N ds = \int_{\Gamma} g_D \bar{\xi}_N = \mathcal{F}\left(\xi_N\right) \quad\forall\xi_N\in V_N
\end{equation*}

La soluzione sarà una sovrapposizione lineare delle funzioni di base dello spazio $V_N$, cioè
\begin{equation*}
\psi_N = \sum_{j=1}^{N} c_j \phi_j
\end{equation*}

E' possibile trovare i coefficienti $c_j$ risolvendo un sistema lineare con matrice dei coefficienti $A^{Gal}_{ij}$ e vettore dei termini noti $F^{Gal}_j$ definiti nel seguente modo
\begin{align*}
A^{Gal}_{ji} & = \int_{\Gamma} \left( S \phi_i \right) \bar{\phi}_j ds\left(\mathbf{x}\right) =%
\int_{\Gamma} \int_{\Gamma} \Phi_k\left(\mathbf{x},\mathbf{y}\right) \phi_i\left(\mathbf{y}\right) \bar{\phi}_j\left(\mathbf{x}\right) ds\left(\mathbf{y}\right)ds\left(\mathbf{x}\right)\\
F^{Gal}_j & = \int_{\Gamma} g_D\left( \mathbf{x} \right) \bar{\phi}_j \left(\mathbf{x}\right)ds\left(\mathbf{x}\right)
\end{align*}


### IGA

Data una Bspline che definisce una curva o una superficie è possibile sfruttare l'approccio detto *IsoGeometric Analysis* per definire su tale curva o superficie delle funzioni: negli approcci FEM o BEM il dominio delle funzioni di base $\Gamma$ diventa una "varietà" parametrizzata dalle Bspline.
Come conseguenza di tale approccio, che ha tra i vantaggi la possibilità di poter rappresentare anche domini $\Gamma$ curvi (alcuni anche in modo esatto), gli integrali che il metodo di Galerkin richiede di calcolare non vengono valutati su $\Gamma$ ma in $\left[0,1\right]$. Questo perché anche le funzioni di base appartenenti a $V_N$ e definite in $\Gamma$ vengono "riscritte" in funzioni Bspline scalari definite in $\left[0,1\right]$. Le funzioni di $V_N$ vengono infatti scelte in modo tale che 
\begin{equation*}
\phi_i \left(\mathbf{x}\right) = \phi_i \left(\mathbf{x}\left(t\right)\right) = \hat{\phi_i}\left(t\right) \quad\text{con}\quad\,t\in  \left[0,1\right]
\end{equation*}
Dunque la funzione $t\rightarrow \mathbf{x}\left(t\right)$ è rappresentata da una curva Bspline `bs` definita in $\left[0,1\right]$ a valori in $\mathbf{R}^2$ mentre le funzioni di base $\hat{\phi_i}\left(t\right)$ sono delle funzioni Bspline scalari, con lo stesso grado polinomiale `P`, con la stessa cardinalità `N`, con lo stesso knot vector `u` di `bs` ma a valori scalari.

Gli integrali precedenti vengono dunque calcolati nel seguente modo
\begin{align*}
A^{Gal}_{ji} & = %
\int_{\left[0,1\right]} \int_{\left[0,1\right]} %
\Phi_k\left(\mathbf{x}\left(t\right),\mathbf{x}\left(s\right)\right) %
\hat{\phi}_i\left(s\right) %
\hat{\phi}_j\left(t\right)  %
\left| \frac{\partial \mathbf{x}}{\partial t} \right| %
\left| \frac{\partial \mathbf{x}}{\partial s} \right| %
dt ds \\
F^{Gal}_j & = %
\int_{\left[0,1\right]} %
g_D\left( \mathbf{x}\left(t\right) \right) %
\hat{\phi}_j \left(t\right)%
\left| \frac{\partial \mathbf{x}}{\partial t} \right|
dt
\end{align*}
Non è stato scritto il simbolo di complesso coniugato perché tanto nel nostro caso le funzioni di base sono tutte a valori reali.

Ovviamente le funzioni di base sono polinomi a tratti, quindi il loro sostegno non è $\left[0,1\right]$ ma un intervallo in esso compreso: nel codice si è tenuto conto di questo fatto per ottimizzare il calcolo degli integrali.

Per quanto riguarda la *representation formula*, l'integrale è stato valutato tramite un approccio IGA ma con una piccola ottimizzazione che sfrutta la linearità della soluzione
\begin{align*}
\left(\mathcal{S}\psi_N\right)\left( \mathbf{x}\right) & = %
\int_{\Gamma} \Phi_k \left(\mathbf{x},\mathbf{y}\right) %
\psi_N \left(\mathbf{y}\right)ds\left(\mathbf{y}\right) %
\quad\text{con}\quad\mathbf{x}\in \Omega_{+}\\
& = \int_{\Gamma} \Phi_k \left(\mathbf{x},\mathbf{y}\right)%
\sum_{j=1}^{N} c_j \phi_j \left(\mathbf{y}\right)ds\left(\mathbf{y}\right)\\
& = \sum_{j=1}^{N} c_j %
\int_{\Gamma} \Phi_k \left(\mathbf{x},\mathbf{y}\right)%
\phi_j \left(\mathbf{y}\right)ds\left(\mathbf{y}\right)\\
& = \sum_{j=1}^{N} c_j %
\int_{\left[0,1\right]} \Phi_k \left(\mathbf{x},\mathbf{y}\left(t\right)\right)%
\hat{\phi}_j \left(t\right) %
\left| \frac{\partial \mathbf{y}}{\partial t} \right| dt\\
& = \sum_{j=1}^{N} c_j  M_j\left(\mathbf{x}\right)
\end{align*}
Dove gli elementi della sommatoria sono stati valutati separatamente per ogni punti $\mathbf{x}$ richiesto, in questo modo variando l'onda incidente $u^{inc}$ (dunque variando i coefficienti $c_j$) non era necessario valutare di nuovo l'integrale appena scritto ma soltanto calcolare la somma dell'ultima riga

## Implementazione

Le principali funzioni della classe `Bspline` necessarie per descrivere funzioni, curve e superfici sono le seguenti:
- `__init__` : costruttore della classe, è necessario specificare il grado polinomiale `P` (in tutto il notebook si sono usate solo Bspline di grado polinomiale `P=1`), il numero di gradi di libertà `N` che coincide con la dimensionalità dello spazio $V_N$ (si faccia attenzione che questo non coincide esattamete con *il numero di funzioni di base Bspline*, questo deriva dal fatto che stiamo usando Bspline periodiche, spiegherò meglio questo dettaglio in [seguito](#periodicity)), il knot vector `v` ed altri parametri di importanza secondaria
- `get_cp` e `set_cp` : per impostare i control points e ottenerne il valore (i control points sono allocati come `ndarray` di `numpy`)
- `control_points` : funzione ausiliaria per vedere *scritti bene* e in modo intuitivo i control points come dataframe di `pandas`
- `basis_range`: restituisce l'effettivo dominio delle varie funzioni di base
- `show`: stampa a schermo le informazioni principali della classe
- `dim` e `codim`: restituisce la dimensionalità del dominio e del codominio, nel nostro caso saranno sempre rispettivamente `dim=1` e `codim=2` (per la Bspline che parametrizza $\Gamma$) e `codim=1` per le funzioni di base
- `evaluate` : funzione per valutare la Bspline in un punto `x`, viene usato l'algoritmo di deBoor, sfrutta altre funzioni di appoggio

Il BEM è invece stato implementato tramite l'omonima funzione `BEM` che chiama in ordine le seguenti funzioni:
- `stiffness_matrix_BEM`: calcola la matrice $A^{Gal}_{ij}$
- `load_vector_BEM`: calcola il vettore $F^{Gal}_{i}$ data la una funzione $g_D$
- `indirect_solution_BEM`: risolve il sistema lineare che deriva dal metodo di Galerkin
- `single_layer_potential_basis_BEM`: calcola gli elementi $M_j\left(\mathbf{x}\right)$ data una lista di punti
- `single_layer_potential_BEM`: calcola la somma $\left(\mathcal{S}\psi_N\right)\left( \mathbf{x}\right) = \sum_{j=1}^{N} c_j  M_j\left(\mathbf{x}\right)$

Altre funzioni importanti e ampiamente usate sono:
- `internal_points`: calcolo l'indice di avvolgimento di un punto per capire se questo è interno o esterno ad una curva
- `derivative`: la derivata di una Bspline di grado `P`, con `N` funzioni di base e knot vector `v` è una Bspline di grado `P-1`, con `N-1` funzioni di base e knot vector `v` a cui sono stati eliminati il primo e l'ultimo knot, questa funzione restituisce dunque una nuova Bspline
- `_scalar`: data la Bspline `bs` che parametrizza il dominio $\Gamma$, questa funzione restituisce una Bspline scalare, cioè una della funzioni $\hat{\phi}_i$ (è sufficiente cambiare i control points con la funzione `set_cp` per passare da una Bspline scalare all'altra)
- `save` e `load`: scrivono/leggono su/da file alcune variabili contenute nella classe, come per esempio i control points, $A^{Gal}_{ij}$, $F^{Gal}_{i}$, i coefficienti $M_j\left(\mathbf{x}\right)$, etc...
- `make_vector_periodic` e `make_matrix_periodic`: il numero di funzioni di *Bspline di base* non coincide con il numero di *gradi di libertà* o delle funzioni di base $\hat{\phi}_i$ (come verrà mostrato in [seguito](#periodicity)), questo perché la periodicità della curva è ottenuta fornzando alcuni control points ad essere uguali tra di loro. Bisogna tener conto di questa cosa nel calcolo della matrice $A^{Gal}_{ij}$ e del vettore $F^{Gal}_{i}$.

# Pacchetti

Carico le due librerie scritte da me:
- `pyBspline`: libreria contenente 3 classi contenuta el file omonimo, la principale è la classe `Bspline` contenente tutti gli algoritmi e funzioni di interesse, le altre due sono le classi `shape` e `knot_vector`
- `FFT`: libreria contenuta el file omonimo per gestire velocemente le trasformate di Fourier e la soluzione analitica dell'equazioen di Helmholtz nel caso di ostacolo circolare

In [ ]:
import pyBspline as Bs
import FFT as esFFT

Carico i pacchetti di Pyhton:
- `numpy`: libreria numerica *di base*, ampiamente usata in tutto il notebook
- `matplotlib`: libreria per fare i grafici, usata per fare qualunque grafico del notebook
- `pandas`: libreria per gestire i dataframe, usata ampiamente in tutto il notebook per gestire i dataframe, per leggerli e scriverli da e su file ed manipolarli. 
- `scipy`: libreria numerica *avanzata*, usata poco
- `os`: *operating system*, libreria per gestire i files
- `re`: libreria per gestire le stringhe con le *regular expression*
- `copy`: libreria per copiare *correttamente* una variabile o una classe

In [ ]:
import numpy as np
from numpy import random

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

import scipy
from scipy.misc import derivative
from scipy.optimize import curve_fit
import scipy.special

import pandas as pd
import copy
import os

import re

#from imp import reload 

Definisco alcune funzioni usata più volte nel notebook per fare alcuni tipi di grafici *ricorrenti*

In [ ]:
###
def norm(x):
    return np.sqrt(np.sum(np.power(x,2.0)))        
      

In [ ]:
#
def plot(fig,n,xB,yB,x,y,c,title,cmap):
    
    ax = fig.add_subplot(n)
    ax.plot(xB, yB, color= "black",label="Bspline")
    sc = ax.scatter(x,y,c=c,cmap=cmap)
    plt.colorbar(sc)
    ax.set_aspect('equal')
    plt.xlim(min(x),max(x))
    plt.ylim(min(y),max(y))
    plt.title(title)
    
    return

In [ ]:
#
def plot_sol_disc(fig,n,xB_arr,yB_arr,x,y,c,title,cmap):
    
    ax = fig.add_subplot(n)
    for xB,yB in zip(xB_arr,yB_arr):
        ax.plot(xB, yB, color= "black",label="Bspline")
    sc = ax.scatter(x,y,c=c,cmap=cmap)
    plt.colorbar(sc)
    ax.set_aspect('equal')
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    plt.title(title)
    
    return

In [ ]:
#
def plot_sol(fig,n,xB,yB,x,y,c,title):
    
    ax = fig.add_subplot(n)
    ax.plot(xB, yB, color= "black",label="Bspline")
    sc = ax.scatter(x,y,c=c,cmap=cmap)
    plt.colorbar(sc)
    ax.set_aspect('equal')
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    plt.title(title)
    
    return

In [ ]:
#
def plot_matrix(sm,file_png=None):
    #
    sm2 = sm.copy()
    sm2["index"] = sm2.index
    #
    new = sm2.melt(id_vars=['index'])# = sm.index
    #
    new2 = new.copy()
    new2["index"] = [ i[0] for i in new2["index"]]
    new2["variable"] = [ i[0] for i in new2["variable"]]
    #new2["value"] = [np.complex(i) for i in new2["value"] ]
    new2["real"] = np.real(new2["value"])
    new2["imag"] = np.imag(new2["value"])
    new2["abs"] = np.absolute(new2["value"])
    new2["phase"] = np.angle(new2["value"])/np.pi
    df = new2
    
    #
    fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

    cmap = 'RdYlBu'

    #
    ax = fig.add_subplot(221)
    sc = ax.scatter(df["index"],df["variable"],c=df["real"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : real")

    #
    ax = fig.add_subplot(222)
    sc = ax.scatter(df["index"],df["variable"],c=df["imag"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : imag")

    #
    ax = fig.add_subplot(223)
    sc = ax.scatter(df["index"],df["variable"],c=df["abs"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : abs")

    #
    ax = fig.add_subplot(224)
    sc = ax.scatter(df["index"],df["variable"],c=df["phase"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : $\\theta / \pi$")

    plt.tight_layout()
    if file_png is not None :
        plt.savefig(file_png)
    plt.show()
    

# Basis function

In questa parte do una rappresentazione grafica delle funzioni di base $\phi_j$ nel caso in cui $\Gamma$ sia una circonferenza. Inoltre cerco di spiegare nel dettaglio come vengono allocati gli oggetti di tipo `Bspline`.

La seguente cella permette di definire una classe `Bspline` nel seguente modo :
- si definisce la variabile `sh` come un oggetto di tipo `shape`, classe che mi permette di definire la dimensionalità del dominio e del codominio della futura Bspline
- si definisce un knot vector tramite la variabile `kv`: ho creato una funzione che mi genera in automatico il knot vector corretto dato il dominio "effettivo" della Bspline, il numero di gradi di libertà e il grado polinomiale
- passo le variabili `sh` e `kv` al costruttore della classe `Bspline` (assieme ad un `dictionary` che mi specifica alcune proprietà, come il fatto che la Bspline dovrà essere periodica) per ottenere la variabile `bs`

In [ ]:
#definisco la Bspline

#shape
sh = Bs.shape(1,2)

#defiisco i knot vector
P=1  #polinomial degree
N=20 #base caridnality

#dominio della funzione
xminBs = 0.0 
xmaxBs = 1.0


#knot vector
kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)
#kv.show()

#alloco la Bspline
bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})

Chiamando la funzione `show` della variabile `kv` è possibile stampare a video tutte le informazioni contenute in tale variabile. Notiamo due cose:
- il valore massimo e minimo assunto dai vari knots non coincide con quelli specificati nella funzione `periodic_kv`, cioè con `xminBs` e `xmaxBs`, ma questo è corretto perché il dominio di una Bspline non coincide con l'intervallo spaziato dal knot vector. 
- il numero di funzioni di base non coincide con il numero di gradi di libertà `N` passato alla funzione `periodic_kv` (`N` infatti vuole rappresentare il numero di gradi di libertà e non il numero di funzioni di base), ma questo è corretto perché andremo ad utilizzare una Bspline periodica e questa è ottenuta da una Bspline "semplice" forzando alcuni control points ad essegue uguali ad altri, andando quindi a ridurre il numero di control points "liberi", cioè il numero di gradi di libertà effettivi.

La funzione `periodic_kv` automatizza il calcolo del knot vector in modo tale che la variabile `kv` restituita vada poi a definire effettivamente una Bspline con `N` gradi di libertà (anche se con un numero maggiore di `N` di funzioni di base) e dominio $\left[\right.$ `xminBs`,`xmaxBs` $\left. \right]$

In [ ]:
kv.show()

Nelle prossime celle vado ad impostare i control points della Bspline `bs` in modo tale che questa vada ad approssimare un cerchio.
Questa figura geometria infatti non è rappresentabile in modo esatto con delle Bspline, per farlo servirebbero delle NURBS, che non sono ancora state implementate: una volta implementare però l'intero codice che implementa il BEM potrà essere usato ugualmente senza alcuna modifica.

Quello che vado a fare infatti è rappresentare un poligono regolare in modo esatto: vado ad impostare come control points i vertici di tale poligono, essengo poi la Bspline di grado polinomiale `P=1` ottengo una linea spezzata (polinomio lineare a tratti) in $\mathbf{R}^2$

In [ ]:
#function
x0 = 0.0
y0 = 0.0
a = 1.0
b = 1.0
def func(t):
    #print(cpz)
    cpx = a*np.cos(2*np.pi*t)+x0#np.random.rand(N)
    cpy = b*np.sin(2*np.pi*t)+y0#np.random.rand(N)
    out = np.zeros(shape=(len(t),2))
    for i in range(len(t)):
        out[i,0] = cpx[i]
        out[i,1] = cpy[i]
    return out

In [ ]:
#control points
t = np.linspace(0,1,N,endpoint=False)
cp = func(t)
for i in range(len(t)):
    #bs._cp[i] = cp[i]
    bs.set_cp(i,cp[i])
cpx = cp[:,0]
cpy = cp[:,1]

<a id="periodicity"></a>
Una volta assegnati i control points è utile visualizzarli (prossima cella):
- si noti che sono $22$, cioè quanti la variabile `kv` indica essere la cardinalità delle funzioni di base
- si noti che però il primo e il secondo control points soo uguali rispettivamente al penultimo e all'ultimo,riducendo così di $2$ il numero di gradi di libertà: i gradi di libertà sono infatti $20$, che è uguale al valore di `N` che avevamo prima specificato
- si noti che ogni control point è un array di lunghezza $2$

Se avessimo desiderato allocare una Bspline di grado polinomiale $2$ anzichè $1$ avremmo dovuto identificare (il codice lo fa in automatico ovviamente) i seguenti control points:
- il primo con il terzultimo
- il secondo con il penultimo
- il terzo con l'ultimo

L'identificazione del primo control points con l'ultimo è sufficiente a garantire che la curva sia chiusa, ma identificare anche altri contol points in base al grado polinomiale garantisce che anche la derivata prima e quelle di ordine superiore (se definite) "si comportino bene" e nel nostro caso è importante perché abbiamo bisogno di valutare anche la derivata prima della funzione.

In [ ]:
bs.control_points().T

Nella seguente cella vado a fare un grafico delle funzioni di base $\phi_j$ sfruttando la funzione `_scalar` ed andando a valutare le varie funzioni $\phi_j$ soltanto nel loto effettivo dominio, che conosco perché me lo restituisce la funzione `basis_range`: può essere utile vederne l'output.

In [ ]:
bs.basis_range().T

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
df = pd.DataFrame(xy)
df = df.rename(columns={0:"x",1:"y"})

#grafico
fig = plt.figure ( 0 , figsize = ( 10, 8 ) )

bsCopy = bs.copy()
bsCopy.clear_cp()
bsScal = bsCopy._scalar()
br = bs.basis_range()

#
#s = 0.2
ax = fig.add_subplot(111, projection='3d')

for i in range(N):
    print(i+1,"/",N,end="\r")
    u = np.linspace(br.at[(i,),("min",0)],br.at[(i,),("max",0)],100)
    #u = np.linspace(0,1,1000,endpoint=True)
    bsCopy.set_cp(i,bs.get_cp(i))
    bsScal.set_cp(i,1.)
    xyB = bs.evaluate(u)
    zB = bsScal.evaluate(u)
    
    #xyB = xyB[zB != 0]
    #zB  = zB[zB != 0]
    
    #xyB,zB = [ i,j for i,j in zip(xyB,zB) if j != 0.0 ]
    
    ax.plot(xyB[:,0], xyB[:,1],zB,label=str(i))
    
    bsCopy.set_cp(i,[0,0])
    bsScal.set_cp(i,0.)
    
print("Finished")

ax.plot(xy[:,0], xy[:,1],0.0,color="red",label="Bspline")
ax.set_zlim(0,1)
plt.grid(True)
#plt.legend()
plt.show()

# Triangle

Per rendere meno dispersivi i commenti, descriverò buona parte del capitolo [Triangle](#Triangle) qui.

In questo capitolo applico il BEM ad un problema di scattering con un ostacolo di forma triangolare: devo dunque impostare i control points della variabile `bs` in modo tale che questa rappresenti un triangolo: di [seguito](#grafico-triangolo) è possibile vedere un grafico della Bspline e dei control points. 

Nella prossima cella vado invece a definire il vettore d'onda del problema:
- definisco il vettore `k_in` che mi servirà poi nella valutazione dell'onda piana incidente sull'ostacolo
- definisco il suo modulo `wavevector=20.0`

Nella sezione [Stiffness matrix](#triangle-sm) vado a calcolare tramite la funzione `stiffness_matrix_BEM` la matrice $A^{Gal}_{ij}$.
Vado poi a graficarne il valore reale, immaginario, il modulo e la fase usando il colore come "indicatore dell'intensità", mentre sull'asse delle x e delle y sono presenti i vari control points: l'immagine sembra un continuo ma in realtà è uno scatterplot formato da punti molto fitti (sono stati usati `N=100` gradi di libertà). 

Questi grafici hanno adesso poca importanza, ma sono stati durante la fase di sviluppo del codice molto importanti per controllare che non vi fossero "discontinuità" nel calcolo di $A^{Gal}_{ij}$: tramite questi grafici è stato possibile infatti controllare che control points "vicini" restituissero elementi di $A^{Gal}_{ij}$ simili e che la continuità di $A^{Gal}_{ij}$ è preservata anche attraversando i bordi del grafico. Si ricordi che stiamo analizzando una curva chiusa, quindi il control points di indice $0$ è in realtà a contatto con quello di indice $99$: la presenza di "discontinuità" nei pressi del bordo destro e del bordo in alto del grafico mostrava *visivamente* la presenza di errori nel codice che sono poi stati corretti.

Nella sezione [Single Layer Potential Basis](#triangle-slp) si va a generare una griglia di punti `XY0` e tramite la funzione `internal_points` si va a calcolare l'indice di avvolgimento dei vari punti scelti: quelli che risultano interni alla curva vengono ovviamente scartati, quelli esterni sono rappresentati nella variabile `XY`.
Nell'ultima cella di questa sezione si chiama la funzione `single_layer_potential_basis_BEM` per calcolare i coefficienti che ho prima indicato con $M_j\left(\mathbf{x}\right)$ per tutti i punti contenuti nella variabile `XY`.

Sia la matrice $A^{Gal}_{ij}$ che i coefficienti $M_j\left(\mathbf{x}\right)$ (contenuti nelle variabii rispettivamente `bs._sm_BEM` e `bs._slp_BEM`) vengono salvati su file tramite la funzione `save`. E' possibile anche leggere queste variabili da file tramite la funzione `load` in modo tale da evitarne di nuovo il calcolo.

Nonostante le funzioni `stiffness_matrix_BEM` e `single_layer_potential_basis_BEM` verrebbero chiamate automaticamente dalla funzione `BEM` (vero cuore dell'approccio BEM) ho scelto di controllarne gli output accuratamente proprio nelle due sezioni: [Stiffness matrix](#triangle-sm) e [Single Layer Potential Basis](#triangle-slp). 

Questo deriva anche dal fatto che il calcolo di queste variabili non dipende dalla funzione incidente $u^{inc}$: $A^{Gal}_{ij}$ e $M_j\left(\mathbf{x}\right)$ verranno usati identici sia nel caso in cui l'onda incidente sia un'[onda piana](#triangle-planewave) che nel caso sia una funzione di [Herglotz](#triangle-herglotz). 

Ovviamente è possibile definire un qualunque altro tipo di onda indicente andando a riscrivere opportunamente una nuova sezione del notebook sulla falsa riga di [Plane wave](#triangle-planewave) o [Herglotz](#triangle-herglotz) senza dover ricalcolare `stiffness_matrix_BEM` e `single_layer_potential_basis_BEM`. 

I commenti relativi alle sezioni [Plane wave](#triangle-planewave) e [Herglotz](#triangle-herglotz) si trovano in tali sezioni.

## Definition

In [ ]:
#definisco il vettor d'onda
k_in = 20*np.asarray([0.5,np.sqrt(3)/2])
wavevector = np.sqrt(np.sum(np.power(k_in,2.0)))
I = np.complex(0,1)

xmin = -1.5
xmax = 1.5
ymin = -1.5
ymax = 1.5

print(wavevector)

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,2)
#sh.show()

#defiisco i knot vector
P=1 #polinomial degree
N=100 #base caridnality
xminBs = 0.0
xmaxBs = 1.0


#
#kv = Bs.uniform_open_kv(xmin,xmax,p=P,n=N)#Bs.knot_vector(P,N,v)
#kv = periodic_kv(xmin,xmax,p=P,n=N)
kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)
#kv.show()

#alloco la Bspline
bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})

In [ ]:
#files
file_dir = "files/BEM/triangle-periodic/"
suffix = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+".csv"
suffix_png = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+".png"

<a id="grafico-triangolo"></a>
## Geometry

In [ ]:
#triangolo
x0 = -0.5
y0 =  -0.5

a = 1.0 / (2+np.sqrt(2))
b = (1.0+np.sqrt(2)) / (2+np.sqrt(2))
delta = b-a

sx = 1.0
sy = 1.0

def triangle_x(i):
    if i <= a :
        return i
    elif i > a and i <= b :
        j = i-a
        return a - j*a/(delta)
    else :
        return 0
    
def triangle_y(i):
    if i <= a :
        return 0
    elif i > a and i <= b :
        j = i-a
        return j*a/(delta)
    else :
        j = i-b
        return a-j
    
def func(t):
    out = np.zeros((len(t),2))
    out[:,0] = [triangle_x(i) for i in t]
    out[:,1] = [triangle_y(i) for i in t]
            
    out[:,0] = sx*out[:,0]/a+x0
    out[:,1] = sy*out[:,1]/a+y0
    return out

In [ ]:
#ATTENZIONE: mi servono dei punti distribuiti in modo uniforme
# per come ho costruito func so che
# func(0.) = func(1.)
# quindi genero un punto in più
t = np.linspace(0,1,N+1,endpoint=True)#[0:-2]
cp = func(t)
for i in range(len(t)):
    #bs._cp[i] = cp[i]
    bs.set_cp(i,cp[i])
cpx = cp[:,0]
cpy = cp[:,1]

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
Txy = func(T)
df = pd.DataFrame(xy)
df = df.rename(columns={0:"x",1:"y"})

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#converto in dataframe    
ax = fig.add_subplot(131)
plt.plot(Txy[:,0],Txy[:,1],color="green",label="triangle",linestyle="--")
plt.scatter(cpx,cpy,color="green",label="cp")
#plt.scatter(df["x"], df["y"], color= "red",label="Bspline")
plt.grid()
plt.legend()
ax.set_aspect('equal')

ax = fig.add_subplot(132)
#plt.plot(cpx,cpy,color="green",label="cp",linestyle="--")
#plt.scatter(cpx,cpy,color="green",label="cp")
plt.scatter(df["x"], df["y"], color= "red",label="Bspline")
plt.grid()
plt.legend()
ax.set_aspect('equal')

#real
ax = fig.add_subplot(133)#, projection='3d')
ax.plot(T,df["x"],color="blue",label="x")
ax.plot(T,df["y"],color="green",label="y")
#ax.plot(df["t"],np.real(df["trace"]),color="red",label="trace")
#plt.title("real")
plt.grid(True)
plt.legend()
plt.show()

<a id="triangle-sm"></a>
## Stiffness Matrix

In [ ]:
#files
file = file_dir+"stiffness_matrix-n=6-random=False-"+suffix
file

In [ ]:
#stiffness matrix
READ = True
SAVE = False
if os.path.exists(file) and READ == True :
    sm = bs.load("sm-BEM",file)
else :
    sm,out = bs.stiffness_matrix_BEM(k=wavevector,\
                opts={"print":True,"N":[6],"return_both":True,"ready_sm_BEM":False,"random":False})
    if SAVE == True :
        bs.save("sm-BEM",file)
sm.head()

In [ ]:
#
def plot_matrix(sm,file_png=None):
    #
    sm2 = sm.copy()
    sm2["index"] = sm2.index
    #
    new = sm2.melt(id_vars=['index'])# = sm.index
    #
    new2 = new.copy()
    new2["index"] = [ i[0] for i in new2["index"]]
    new2["variable"] = [ i[0] for i in new2["variable"]]
    #new2["value"] = [np.complex(i) for i in new2["value"] ]
    new2["real"] = np.real(new2["value"])
    new2["imag"] = np.imag(new2["value"])
    new2["abs"] = np.absolute(new2["value"])
    new2["phase"] = np.angle(new2["value"])/np.pi
    df = new2
    
    #
    fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

    cmap = 'RdYlBu'

    #
    ax = fig.add_subplot(221)
    sc = ax.scatter(df["index"],df["variable"],c=df["real"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : real")

    #
    ax = fig.add_subplot(222)
    sc = ax.scatter(df["index"],df["variable"],c=df["imag"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : imag")

    #
    ax = fig.add_subplot(223)
    sc = ax.scatter(df["index"],df["variable"],c=df["abs"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : abs")

    #
    ax = fig.add_subplot(224)
    sc = ax.scatter(df["index"],df["variable"],c=df["phase"],cmap = 'RdYlBu')
    plt.colorbar(sc)
    plt.xlim(min(df["index"]),max(df["index"]))
    plt.ylim(min(df["index"]),max(df["index"]))
    ax.set_aspect('equal')
    plt.title("stiffness matrix : $\\theta / \pi$")

    plt.tight_layout()
    if file_png is not None :
        plt.savefig(file_png)
    plt.show()
    

In [ ]:
#grafico
ile_png = file_dir+"stiffness_matrix-n=6-random=False-"+suffix_png
plot_matrix(sm,file_png)

<a id="triangle-slp"></a>
## Single Layer Potential basis

In [ ]:
#files
file = file_dir+"single_layer_potential-"+suffix
file

In [ ]:
#punti XY
Nx= int(xmax-xmin)*20
Ny = int(ymax-ymin)*20
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

#tolgo elementi interni
#radius = np.asarray([np.sqrt(np.sum(np.power(i,2.0))) for i in XY])
internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]

In [ ]:
#single layer potentail per le funzioni di base
READ = True
SAVE = True
if os.path.exists(file) and READ == True :
    slp = bs.load("slp-BEM",file)
#else :

# I can update it adding some  points
slp = bs.single_layer_potential_basis_BEM(XY=XY,k=wavevector,\
                                              opts={"print":True,"N":[6]})
if SAVE == True :
    bs.save("slp-BEM",file)
    
slp.head()

<a id="triangle-planewave"></a>
## Plane wave

Scelgo come onda incidente un'onda piana con un vettor d'onda rappresentato dalla varaibile `k_in`, per fare ciò definisco la funzione `plane_wave`. 
Vado poi a graficarne la traccia $\gamma^{+}u^{inc}$ in $\Gamma$ e il valore (parte reale a sinistra e immaginaria a destra) che questa assume in $\Omega^{+}$.

Nella sottosezione [Solution](#triangle-BEM") richiamo la funzione `BEM`: sfrutto il metodo di Galerkin e poi sfrutto la *representation formula* per ricavare la soluzione del EDP. 

Salvo poi su file per usi futuri la soluzione $\psi_N$ (`bs.save("ind_sol-BEM",file_ind`), il vettore $F^{Gal}_j$ (`bs.save("lv-BEM",file_lv)`) e la soluzione $u^{scat}$ trovata (`bs.save("sol-BEM",file_sol)`).

Per trovare la soluzione del SSSP devo sommare all'onda incidente $u^{inc}$ l'onda generata dall'ostacolo $u^{scat}$: nell'ultimo grafico della sezione sono graficati $u^{inc}$,$u^{scat}$ e $u^{tot}$

### Preparation

In [ ]:
#punti XY
Nx= int(xmax-xmin)*20
Ny = int(ymax-ymin)*20
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

#tolgo elementi interni
#radius = np.asarray([np.sqrt(np.sum(np.power(i,2.0))) for i in XY])
internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]

In [ ]:
#plane wave
def plane_wave(xx): # soluzione
    xx = np.asarray(xx)
    theta = np.dot(xx,k_in)
    return np.exp(I*theta)

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
df = pd.DataFrame(xy)
df = df.rename(columns={0:"x",1:"y"})
uinc = plane_wave(xy)

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#
#s = 0.2
ax = fig.add_subplot(121, projection='3d')
ax.plot(xy[:,0], xy[:,1],uinc.real,color="blue",label="real")
ax.plot(xy[:,0], xy[:,1],uinc.imag,color="green",label="imag")
ax.plot(xy[:,0], xy[:,1],0.0,color="red",label="Bspline")
plt.grid(True)
plt.legend()

#
ax = fig.add_subplot(122)#, projection='3d')
ax.plot(T,uinc.real,color="blue",label="real")
ax.plot(T,uinc.imag,color="green",label="imag")
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

Uinc = plane_wave(XY)

cmap = 'RdYlBu'
    
plot(fig,121,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
plot(fig,122,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)

plt.show()

<a id="triangle-BEM"></a>
### Solution

In [ ]:
#files
file_sol = file_dir+"solution-plane_wave-"+suffix
file_lv  = file_dir+"load_vector-plane_wave-"+suffix
file_ind = file_dir+"indirect_solution-plane_wave-"+suffix
print(file_sol)
print(file_lv)
print(file_ind)

In [ ]:
#metodo di Galerkin
READ = False
SAVE = True
if os.path.exists(file_sol) and READ == True :
    sol,Xnp,Valnp = bs.load("sol-BEM",file_sol)
    
if os.path.exists(file_lv) and READ == True :
    lv = bs.load("lv-BEM",file_lv)
    
if os.path.exists(file_ind) and READ == True :
    sol = bs.load("ind_sol-BEM",file_ind)
    
else :
    opts = {"print":True,"ready_sol_BEM":False,"ready_lv_BEM":False,"ready_ind_sol_BEM":False}
    sol,Xnp,Valnp = bs.BEM(uinc=plane_wave,k=wavevector,XY=XY,opts=opts)
    if SAVE == True :
        bs.save("sol-BEM",file_sol)
        bs.save("lv-BEM",file_lv)
        bs.save("ind_sol-BEM",file_ind)
sol.head()

In [ ]:
#uinc
Uinc = plane_wave(XY)#.reshape(Nx,Ny).transpose()
total = Uinc + Valnp

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'

    
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs")

plt.tight_layout()

sol_png = file_dir+"solution-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

<a id="triangle-herglotz"></a>
## Herglotz

Questa sezione è analoga alla [precedente](#triangle-planewave), con la sola differenza che ho scelto come onda incidente una funzione di Herglotz anzichè un'onda piana.

La prima parte di questa sezione ha l'obiettivo di definire il kernel $g\left(\theta\right)$ della funzione di Herglotz. Ho scelto (più o meno) lo stesso kernel presente nelle dispense in modo tale da confrontare i risultati

$g\left(\theta\right) = \[   \left\{
\begin{array}{ll}
      1 & \pi/6 \leq \theta \leq  \pi/3 \\
      0 & \text{otherwise}
\end{array} 
\right. \]$

Il kernel assume un valore pari ad $1$ soltanto in uno "spicchio" di $1/12$ di angolo giro: risulta comodo definire il kernel come una Bspline periodica (perché $\theta$ è periodico), scalare, di grado polinomiale `P=0` (funzione costante a tratti) con `N=12` funzioni di base, con tutti i control points (scalari) nulli tranne il secondo, cioè quello con indice $1$. 

Il seguente grafico mostra la Bspline che rappresenta il kernel.
Ho poi definito la funzione `Herglotz` che ha come parametri di input soltanto il punto in cui la si vuole valutare  (questa è la "sostituta" di quella che prima era la funzione `plane_wave`), che richiama però a suo volta la funzione `Herglotz_private`, che sarebbe una funzione di Herglotz generica che ha come input anche il vettor d'onda, il kernel (che sono però fissi per la funzione `Herglotz` perché lo sono anche nel nostro problema) e anche il numero di punti con cui si vuole valutare l'integrale in $\theta$.
Nel grafico successivo ho mostrato la parte reale, immaginaria e il modulo della funzione di Herglotz scelta.

Le sottosezioni [Preparation](#triangle-herglotz-preparation) e [Solution](#triangle-herglotz-solution) sono analoghe a quelle della sezione [precedente](#triangle-planewave).

### Kernel

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,1)
#sh.show()

#defiisco i knot vector
P=0 #polinomial degree
N=12 #base caridnality

#
#kv = Bs.uniform_open_kv(xmin,xmax,p=P,n=N)#Bs.knot_vector(P,N,v)
#kv = periodic_kv(xmin,xmax,p=P,n=N)
kv = Bs.periodic_kv(0.0,2*np.pi,p=P,n=N)
#kv.show()

#alloco la Bspline
kernel = Bs.Bspline(sh,[kv],properties={"periodic":[True],"dtype":np.complex})
#bs.show()

kernel.clear_cp()
kernel.set_cp(1,1)

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(0.0,2*np.pi,NN,endpoint=False)
y   = kernel.evaluate(T)
#df = pd.DataFrame(xy)
#df = df.rename(columns={0:"x",1:"y"})

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#real
ax = fig.add_subplot(111)#, projection='3d')
ax.plot(T,y.real,color="blue",label="real")
ax.plot(T,y.imag,color="green",label="imag")
plt.xlabel(r"$\theta \, \left[ rad \right]$")
plt.title(r"Herglotz kernel $\, g \left( \theta \right) \, : 1 \, $ if $ \,  - \pi/6 < \theta < \pi/6$")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Herglotz
def Herglotz_private(xy,kernel,k,NN):
    xy = np.asarray(xy)
    theta = np.linspace(0.,2*np.pi,NN,endpoint=False)
    cos = np.cos(theta)
    sin = np.sin(theta)
    g = kernel(theta)
    phase = np.outer(xy[:,0],cos) + np.outer(xy[:,1],sin)
    expo = np.exp(1.j*phase*k)
    return np.dot(expo,g)/NN

In [ ]:
#k = 30./4.
NN = 100
def Herglotz(xy): 
    return Herglotz_private(xy,kernel,wavevector,NN)

In [ ]:
#punti XY
Nx= int(xmax-xmin)*20
Ny = int(ymax-ymin)*20
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY = np.zeros((Nx*Ny,2))
XY[:,0] = X.reshape((Nx*Ny,))
XY[:,1] = Y.reshape((Nx*Ny,))

x = XY[:,0]
y = XY[:,1]

In [ ]:
Uinc = Herglotz(XY)

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

cmap = 'RdYlBu'

ax = fig.add_subplot(131)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=Uinc.real,cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("real")

ax = fig.add_subplot(132)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=Uinc.imag,cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("imag")

ax = fig.add_subplot(133)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=np.absolute(Uinc),cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("abs")

#ax = fig.add_subplot(224)
##ax.plot(xB, yB, color= "black",label="Bspline")
#sc = ax.scatter(x,y,c=np.angle(Uinc),cmap=cmap)
#plt.colorbar(sc)
#ax.set_aspect('equal')
#plt.xlim(min(x),max(x))
#plt.ylim(min(y),max(y))
#plt.title("phase")

plt.show()

<a id="triangle-herglotz-preparation"></a>
### Preparation

In [ ]:
#punti XY
Nx= int(xmax-xmin)*20
Ny = int(ymax-ymin)*20
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
uinc = Herglotz(xy)

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#
#s = 0.2
ax = fig.add_subplot(121, projection='3d')
ax.plot(xy[:,0], xy[:,1],uinc.real,color="blue",label="real")
ax.plot(xy[:,0], xy[:,1],uinc.imag,color="green",label="imag")
ax.plot(xy[:,0], xy[:,1],0.0,color="red",label="Bspline")
plt.grid(True)
plt.legend()

#
ax = fig.add_subplot(122)#, projection='3d')
ax.plot(T,uinc.real,color="blue",label="real")
ax.plot(T,uinc.imag,color="green",label="imag")
plt.grid(True)
plt.legend()

#
#ax = fig.add_subplot(133)#, projection='3d')
#ax.plot(T,somma.real,color="blue",label="real")
#ax.plot(T,somma.imag,color="green",label="imag")
#plt.grid(True)
#plt.legend()

plt.show()

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

Uinc = Herglotz(XY)

cmap = 'RdYlBu'
    
plot(fig,121,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
plot(fig,122,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)

plt.show()

<a id="triangle-herglotz-solution"></a>
### Solution

In [ ]:
#files
file_sol = file_dir+"solution-Herglotz-"+suffix
file_lv  = file_dir+"load_vector-Herglotz-"+suffix
file_ind = file_dir+"indirect_solution-Herglotz-"+suffix
print(file_sol)
print(file_lv)
print(file_ind)

In [ ]:
#metodo di Galerkin
READ = False
SAVE = True
if os.path.exists(file_sol) and READ == True :
    sol,Xnp,Valnp = bs.load("sol-BEM",file_sol)
    
if os.path.exists(file_lv) and READ == True :
    lv = bs.load("lv-BEM",file_lv)
    
if os.path.exists(file_ind) and READ == True :
    sol = bs.load("ind_sol-BEM",file_ind)
    
else :
    opts = {"print":True,"ready_sol_BEM":False,"ready_lv_BEM":False,"ready_ind_sol_BEM":False}
    sol,Xnp,Valnp = bs.BEM(uinc=Herglotz,k=wavevector,XY=XY,opts=opts)
    if SAVE == True :
        bs.save("sol-BEM",file_sol)
        bs.save("lv-BEM",file_lv)
        bs.save("ind_sol-BEM",file_ind)
sol.head()

In [ ]:
#uinc
Uinc = Herglotz(XY)#.reshape(Nx,Ny).transpose()
total = Uinc + Valnp

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'

    
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs")

plt.tight_layout()

sol_png = file_dir+"solution-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

# Circle

Questa sezione è analoga a [Triangle](#Triangle), l'unica aggiunta sono le sottosezioni *Analytic solution* per l'[onda piana](#analytic-planewave) e per la funzione di [herglotz](#analytic-herglotz): ho inserito dei commenti solo nella parte relativa all'[onda piana](#analytic-planewave), quella con la funzione di [herglotz](#analytic-herglotz) è analoga.

## Definition

In [ ]:
#definissco il vettor d'onda
k_in = 30/4*np.asarray([np.sqrt(3.)/2.,0.5])
wavevector = 7.5#np.sqrt(np.sum(np.power(k_in,2.0)))
print("w:",wavevector)
print("h:",np.pi/(5*wavevector))
I = np.complex(0,1)

xmin = -3
xmax = 3
ymin = -3
ymax = 3

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,2)
#sh.show()

#defiisco i knot vector
P=1 #polinomial degree
N=100 #base caridnality
xminBs = 0.0
xmaxBs = 1.0


#
#kv = Bs.uniform_open_kv(xmin,xmax,p=P,n=N)#Bs.knot_vector(P,N,v)
#kv = periodic_kv(xmin,xmax,p=P,n=N)
kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)
#kv.show()

#alloco la Bspline
bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})

In [ ]:
#files
file_dir = "files/BEM/circle/"
suffix = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+".csv"
suffix_png = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+".png"

## Geometry

In [ ]:
#geometria
x0 = 0.0
y0 = 0.0
a = 1.0
b = 1.0
radius = 1.0
def func(t):
    #print(cpz)
    cpx = a*np.cos(2*np.pi*t)+x0#np.random.rand(N)
    cpy = b*np.sin(2*np.pi*t)+y0#np.random.rand(N)
    out = np.zeros(shape=(len(t),2))
    for i in range(len(t)):
        out[i,0] = cpx[i]
        out[i,1] = cpy[i]
    return out

In [ ]:
#ATTENZIONE: mi servono dei punti distribuiti in modo uniforme
# per come ho costruito func so che
# func(0.) = func(1.)
# quindi genero un punto in più
t = np.linspace(0,1,N+1,endpoint=True)#[0:-2]
cp = func(t)
for i in range(len(t)):
    #bs._cp[i] = cp[i]
    bs.set_cp(i,cp[i])
cpx = cp[:,0]
cpy = cp[:,1]

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
df = pd.DataFrame(xy)
df = df.rename(columns={0:"x",1:"y"})

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#converto in dataframe    
ax = fig.add_subplot(131)
#plt.plot(cpx,cpy,color="green",label="cp",linestyle="--")
plt.scatter(cpx,cpy,color="green",label="cp")
#plt.scatter(df["x"], df["y"], color= "red",label="Bspline")
plt.grid()
plt.legend()
ax.set_aspect('equal')

ax = fig.add_subplot(132)
#plt.plot(cpx,cpy,color="green",label="cp",linestyle="--")
#plt.scatter(cpx,cpy,color="green",label="cp")
plt.scatter(df["x"], df["y"], color= "red",label="Bspline")
plt.grid()
plt.legend()
ax.set_aspect('equal')

#real
ax = fig.add_subplot(133)#, projection='3d')
ax.plot(T,df["x"],color="blue",label="x")
ax.plot(T,df["y"],color="green",label="y")
#ax.plot(df["t"],np.real(df["trace"]),color="red",label="trace")
#plt.title("real")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
filename = file_dir+"control_points-"+suffix
a = bs.save("cp",filename)
#bs.load("cp",filename)

## Stiffness Matrix

In [ ]:
#stiffness matrix
READ = True
SAVE = False
file = file_dir+"stiffness_matrix-n=6-"+suffix
print(file)

if os.path.exists(file) and READ == True :
    sm = bs.load("sm-BEM",file)
else :
    sm,out = bs.stiffness_matrix_BEM(k=wavevector,\
                                 opts={"print":True,"N":[6],"ready_sm_BEM":False,"return_both":True})
    if SAVE == True :
        bs.save("sm-BEM",file)
sm.head()

In [ ]:
#grafico
file_png = file_dir+"stiffness_matrix-n=6-random=False-"+suffix_png
plot_matrix(sm,file_png)

## Single Layer Potential basis

In [ ]:
#punti XY
Nx= int(xmax-xmin)*10
Ny = int(ymax-ymin)*10
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]
print(len(XY))#," = ",len(XYslp)/3600,"h")

#tolgo elementi interni
#radius = np.asarray([np.sqrt(np.sum(np.power(i,2.0))) for i in XY])
#XYslp = XY#[radius > 1.0]
#print(len(XYslp)," = ",len(XYslp)*3/3600,"h")

In [ ]:
#single layer potential per le funzioni di base
READ = True
SAVE = True

file = file_dir+"single_layer_potential-"+suffix
print(file)

if os.path.exists(file) and READ == True :
    slp = bs.load("slp-BEM",file)
#else :

# I can update it adding some  points
slp = bs.single_layer_potential_basis_BEM(XY=XY,k=wavevector,\
                                              opts={"print":True,"N":[6]})
if SAVE == True :
    bs.save("slp-BEM",file)
    
slp.head()

## Plane wave

### Preparation

In [ ]:
#punti XY
Nx= int(xmax-xmin)*10
Ny = int(ymax-ymin)*10
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]

In [ ]:
#plane_wave
def plane_wave(xx): # soluzione
    xx = np.asarray(xx)
    theta = np.dot(xx,k_in)
    return np.exp(I*theta)

In [ ]:
#valutazione della Bspline
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
uinc = plane_wave(xy)

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#
#s = 0.2
ax = fig.add_subplot(121, projection='3d')
ax.plot(xy[:,0], xy[:,1],uinc.real,color="blue",label="real")
ax.plot(xy[:,0], xy[:,1],uinc.imag,color="green",label="imag")
ax.plot(xy[:,0], xy[:,1],0.0,color="red",label="Bspline")
plt.grid(True)
plt.legend()

#
ax = fig.add_subplot(122)#, projection='3d')
ax.plot(T,uinc.real,color="blue",label="real")
ax.plot(T,uinc.imag,color="green",label="imag")
plt.grid(True)
plt.legend()

#Mie series


#
#ax = fig.add_subplot(133)#, projection='3d')
#ax.plot(T,somma.real,color="blue",label="real")
#ax.plot(T,somma.imag,color="green",label="imag")
#plt.grid(True)
#plt.legend()

plt.show()

In [ ]:
#uinc
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

Uinc = plane_wave(XY)

cmap = 'RdYlBu'
    
plot(fig,121,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
plot(fig,122,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)

plt.show()

### Solution

In [ ]:
#metodo di Galerkin
READ = True
SAVE = False

#
file_sol = file_dir+"solution-plane_wave-"+suffix
file_lv  = file_dir+"load_vector-plane_wave-"+suffix
file_ind = file_dir+"indirect_solution-plane_wave-"+suffix
print(file_sol)
print(file_lv)
print(file_ind)

if os.path.exists(file_sol) and READ == True :
    sol,Xnp,Valnp = bs.load("sol-BEM",file_sol)
    
if os.path.exists(file_lv) and READ == True :
    lv = bs.load("lv-BEM",file_lv)
    
if os.path.exists(file_ind) and READ == True :
    sol = bs.load("ind_sol-BEM",file_ind)
    
else :
    opts = {"print":True,"ready_sol_BEM":False,"ready_lv_BEM":False,"ready_ind_sol_BEM":False}
    sol,Xnp,Valnp = bs.BEM(uinc=plane_wave,k=wavevector,XY=XY,opts=opts)
    if SAVE == True :
        bs.save("sol-BEM",file_sol)
        bs.save("lv-BEM",file_lv)
        bs.save("ind_sol-BEM",file_ind)
sol.head()

In [ ]:
#uinc
Uinc = plane_wave(XY)#.reshape(Nx,Ny).transpose()
total = Uinc + Valnp

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'

   
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs")

plt.tight_layout()

sol_png = file_dir+"solution-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

<a id="analytic-planewave"></a>
### Analytic solution 

Se $\Gamma$ è una circonferenza è possibile calcolare analiticamente la soluzione cercata: nel file `FFT` ci sono due funzioni, di cui una si chiama `analytic_solution_circle` e ha comme obiettivo proprio quello di calcolare la soluzione analitica $u^{scat}$, i parametri di ingresso di questa funzione sono:
- un vettore `uinc` contenente i valori assunti dalla funzione incidente su $\Gamma$
- i punti `XY` in cui voglio calcolare la soluzione analitica
- `wmin` e `wmax`: per calcolare la soluzione analitica è necessatio calcolare la trasformata di Fourier di `uinc`, questi due parametri impongono di andare a considerare solo un range di frequenze per calcolare la soluzione analitica
- `radius`: raggio della circonferenza
- `wavevector`: vettor'onda del problema
- alcuni parametri aggiunti opzionali

Di seguito grafico il valore che l'onda incidente assume su $\Gamma$.
Si noti che l'asse delle $x$ è ristretto tra $0$ e $1$ perché $\Gamma$ è parametrizzato da una Bspline con dominio $\left[0,1\right]$: dato il valore che $u^{inc}$ assume in un punto qualunque di $\Gamma$ sono in grado di risalive al valore di $t\in\left[0,1\right]$ corrispondente. Nel grafico di destra è mostrata invece la trasformata di Fourier della funzione incidente, mentre nel grafico in basso uno "zoom" di questa in modo tale da vedere meglio i  modi principali della trasformata in modo da poi impostare `wmin` e `wmax` adeguatamente.

Successivamente vado a richiamare la funzione `analytic_solution_circle` e ne stampo a schermo parte dell'output.

Vado poi a graficare la soluzione $u^{Galerkin}$ calcolata numericamente nelle sezioni precedenti, quella calcolata analiticamente $u^{Analytic}$ e la loro differenza. 

Il grafico mostrato risulta però poco significativo se preso a sè: più avanti nel notebook eseguirò uno studio più dettagliato della "differenza" tra la soluzione analitica e quella numerica: si veda [Convergence](#Convergence)

In [ ]:
#
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
uinc = plane_wave(xy)

In [ ]:
#
fft = esFFT.FFT(uinc,opts={"plot":True})

#
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )
ax = fig.add_subplot(111)
plt.plot(fft.index, np.real(fft["fft"]),color="blue" ,label="real")#,marker="+")
plt.plot(fft.index, np.imag(fft["fft"]),color="green",label="imag")#,marker="x")
plt.xlim(-20,20)
plt.legend()
plt.grid(True)
plt.title("Fourier Transform")
plt.show()
#
fft.head()

In [ ]:
#
out,analytic = esFFT.analytic_solution_circle(uinc,XY,wmin=-15,wmax=15,radius=radius,\
                                              wavevector=wavevector,opts={"return":"both"})

analytic_tot = Uinc + analytic

out.head()

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'
    
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}^{Galerkin}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}^{Galerkin}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}^{Galerkin}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],analytic.real,"$u_{scat}^{analytic}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],analytic.imag,"$u_{scat}^{analytic}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(analytic),"$u_{scat}^{analytic}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real-analytic.real,\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag-analytic.imag,\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp-analytic),\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : abs")


plt.tight_layout()

sol_png = file_dir+"solution-analytic-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

## Herglotz

### Kernel

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,1)
#sh.show()

#defiisco i knot vector
P=0 #polinomial degree
N=12 #base caridnality

#
#kv = Bs.uniform_open_kv(xmin,xmax,p=P,n=N)#Bs.knot_vector(P,N,v)
#kv = periodic_kv(xmin,xmax,p=P,n=N)
kv = Bs.periodic_kv(0.0,2*np.pi,p=P,n=N)
#kv.show()

#alloco la Bspline
kernel = Bs.Bspline(sh,[kv],properties={"periodic":[True],"dtype":np.complex})
#bs.show()

kernel.clear_cp()
kernel.set_cp(0,1)

In [ ]:
#
NN = 1000
T = np.linspace(0.0,2*np.pi,NN,endpoint=False)
y   = kernel.evaluate(T)
#df = pd.DataFrame(xy)
#df = df.rename(columns={0:"x",1:"y"})

In [ ]:
#
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#real
ax = fig.add_subplot(111)#, projection='3d')
ax.plot(T,y.real,color="blue",label="real")
ax.plot(T,y.imag,color="green",label="imag")
plt.xlabel(r"$\theta \, \left[ rad \right]$")
plt.title(r"Herglotz kernel $\, g \left( \theta \right) \, : 1 \, $ if $ \,  - \pi/6 < \theta < \pi/6$")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#
def Herglotz_private(xy,kernel,k,NN):
    xy = np.asarray(xy)
    theta = np.linspace(0.,2*np.pi,NN,endpoint=False)
    cos = np.cos(theta)
    sin = np.sin(theta)
    g = kernel(theta)
    phase = np.outer(xy[:,0],cos) + np.outer(xy[:,1],sin)
    expo = np.exp(1.j*phase*k)
    return np.dot(expo,g)/NN

In [ ]:
#k = 30./4.
NN = 100
def Herglotz(xy): 
    return Herglotz_private(xy,kernel,wavevector,NN)

In [ ]:
#
Nx= int(xmax-xmin)*10
Ny = int(ymax-ymin)*10
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY = np.zeros((Nx*Ny,2))
XY[:,0] = X.reshape((Nx*Ny,))
XY[:,1] = Y.reshape((Nx*Ny,))

x = XY[:,0]
y = XY[:,1]

In [ ]:
Uinc = Herglotz(XY)

In [ ]:
#
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

cmap = 'RdYlBu'

ax = fig.add_subplot(131)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=Uinc.real,cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("real")

ax = fig.add_subplot(132)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=Uinc.imag,cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("imag")

ax = fig.add_subplot(133)
#ax.plot(xB, yB, color= "black",label="Bspline")
sc = ax.scatter(x,y,c=np.absolute(Uinc),cmap=cmap)
plt.colorbar(sc)
ax.set_aspect('equal')
plt.xlim(min(x),max(x))
plt.ylim(min(y),max(y))
plt.title("abs")

#ax = fig.add_subplot(224)
##ax.plot(xB, yB, color= "black",label="Bspline")
#sc = ax.scatter(x,y,c=np.angle(Uinc),cmap=cmap)
#plt.colorbar(sc)
#ax.set_aspect('equal')
#plt.xlim(min(x),max(x))
#plt.ylim(min(y),max(y))
#plt.title("phase")

plt.show()

### Preparation

In [ ]:
#
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
uinc = Herglotz(xy)

In [ ]:
#
Nx= int(xmax-xmin)*10
Ny = int(ymax-ymin)*10
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY = np.zeros((Nx*Ny,2))
XY[:,0] = X.reshape((Nx*Ny,))
XY[:,1] = Y.reshape((Nx*Ny,))

internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]

x = XY[:,0]
y = XY[:,1]

In [ ]:
#
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )

#
#s = 0.2
ax = fig.add_subplot(121, projection='3d')
ax.plot(xy[:,0], xy[:,1],uinc.real,color="blue",label="real")
ax.plot(xy[:,0], xy[:,1],uinc.imag,color="green",label="imag")
ax.plot(xy[:,0], xy[:,1],0.0,color="red",label="Bspline")
plt.grid(True)
plt.legend()

#
ax = fig.add_subplot(122)#, projection='3d')
ax.plot(T,uinc.real,color="blue",label="real")
ax.plot(T,uinc.imag,color="green",label="imag")
plt.grid(True)
plt.legend()

#
#ax = fig.add_subplot(133)#, projection='3d')
#ax.plot(T,somma.real,color="blue",label="real")
#ax.plot(T,somma.imag,color="green",label="imag")
#plt.grid(True)
#plt.legend()

plt.show()

In [ ]:
#
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )

Uinc = Herglotz(XY)

cmap = 'RdYlBu'
    
plot(fig,121,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
plot(fig,122,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)

plt.show()

### Solution

In [ ]:
#
READ = True
SAVE = False

file_sol = file_dir+"solution-Herglotz-"+suffix
file_lv  = file_dir+"load_vector-Herglotz-"+suffix
file_ind = file_dir+"indirect_solution-Herglotz-"+suffix
print(file_sol)
print(file_lv)
print(file_ind)

if os.path.exists(file_sol) and READ == True :
    sol,Xnp,Valnp = bs.load("sol-BEM",file_sol)
    
if os.path.exists(file_lv) and READ == True :
    lv = bs.load("lv-BEM",file_lv)
    
if os.path.exists(file_ind) and READ == True :
    sol = bs.load("ind_sol-BEM",file_ind)
    
else :
    opts = {"print":True,"ready_sol_BEM":False,"ready_lv_BEM":False,"ready_ind_sol_BEM":False}
    sol,Xnp,Valnp = bs.BEM(uinc=Herglotz,k=wavevector,XY=XY,opts=opts)
    if SAVE == True :
        bs.save("sol-BEM",file_sol)
        bs.save("lv-BEM",file_lv)
        bs.save("ind_sol-BEM",file_ind)
sol.head()

In [ ]:
#
Uinc = Herglotz(XY)#.reshape(Nx,Ny).transpose()
total = Uinc + Valnp

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'
    
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs")

plt.tight_layout()

sol_png = file_dir+"solution-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

<a id="analytic-herglotz"></a>
### Analytic solution

Commenti analoghi [qui](#analytic-planewave)

In [ ]:
#
NN = 1000
T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
xy   = bs.evaluate(T)
uinc = Herglotz(xy)

In [ ]:
#
fft = esFFT.FFT(uinc,opts={"plot":True})

#
fig = plt.figure ( 0 , figsize = ( 15 , 5 ) )
ax = fig.add_subplot(111)
plt.plot(fft.index, np.real(fft["fft"]),color="blue" ,label="real")#,marker="+")
plt.plot(fft.index, np.imag(fft["fft"]),color="green",label="imag")#,marker="x")
plt.xlim(-20,20)
plt.legend()
plt.grid(True)
plt.title("Fourier Transform")
plt.show()
#
fft.head()

In [ ]:
#
out,analytic = esFFT.analytic_solution_circle(uinc,XY,wmin=-15,wmax=15,radius=radius,\
                                              wavevector=wavevector,opts={"return":"both"})

analytic_tot = Uinc + analytic

out.head()

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )

cmap = 'RdYlBu'
    
plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}^{Galerkin}$ : real")
plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}^{Galerkin}$ : imag")
plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}^{Galerkin}$ : abs")

plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],analytic.real,"$u_{scat}^{analytic}$ : real")
plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],analytic.imag,"$u_{scat}^{analytic}$ : imag")
plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(analytic),"$u_{scat}^{analytic}$ : abs")

plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real-analytic.real,\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : real")
plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag-analytic.imag,\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : imag")
plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp-analytic),\
         "$u_{scat}^{Galerkin}-u_{scat}^{analytic}$ : abs")


plt.tight_layout()

sol_png = file_dir+"solution-analytic-n=6-random=False-"+suffix_png
plt.savefig(sol_png)

plt.show()

# Convergence

In questa sezione vado a calcolare la norma $L^2\left(\Omega_{+}\right)$ della differenza tra la soluzione analitica come calcolata nelle sezioni [Plane Wave](#analytic-planewave) e [Herglotz](#analytic-herglotz) al variare del numero di gradi di libertà usato `N` e del numero di punti usati (indicato con la variabile `opts["N"]`) per calcolare gli integrali.

Nella sottosezione [Cycle](#convergence-cycle) vado a ripercorrere tutto quello che è stato fatto nella sezione [Circle](#Circle):
- variando di volta in volta i gradi di libertà `N=10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,100`
- variando il numero di punti usati per calcolare gli integrali `opts["N"]=4,6,8`
- alloco dunque la Bspline, imposto i control points e determino quali dei punti `XY0` sono interni alla curva
- richiamo la funzione `BEM`, all'interno della quale vengono richiamate automaticamente le funzioni `stiffness_matrix_BEM` e `single_layer_potential_basis_BEM` che prima richiamavo a parte
- viene calcolata la soluzione numerica tramite il metodo BEM e viene salvata nella variabile `sol`
- nella variabile `grafico`, che poi salvo su file, inserisco i valori assunti da $u^{inc}$, $u^{scat}$ e $u^{tot}$ per i vari punti `XY`
- grafico il risultato e lo salvo su file: tutti i grafici sono presenti nella cartella `files/BEM/circle-convergence/` con l'estensionne `png`, assieme a tutti gli altri file contenenti le variabili calcolate ($A^{Gal}_{ij}$,$M_j\left(\mathbf{x}\right)$,etc...)

Nella sottosezione [Norm](#convergence-norm) vado a leggere da file le grandezze calcolate in precedenza, calcolo la soluzione analitica per tutti i valori di `N` e `opts["N"]` usati in precedenza e vado a calcolarne la differenza con la soluzione numerica, per poi vautarne la norma $L^2$, o meglio
- nel dataframe `LN` inserisco la media su tutti i punti `XY` di $\left| u_{scat}^{Galerkin}\left(\mathbf{x}\right) - u_{scat}^{Analytic} \left(\mathbf{x}\right)\right|^2$
- nel datafrem `LNv` inserisco la varianza di $\left| u_{scat}^{Galerkin}\left(\mathbf{x}\right) - u_{scat}^{Analytic} \left(\mathbf{x}\right)\right|^2$

Nella sottosezione [Grafico](#convergence-grafico) vado a graficare con un errorbar `LN` e `LNv` (le barre di errore sono poco visibili):
- sull'asse delle x è rappresentato `N`
- sull'asse delle y è rappresentato `LN`, cioè $\left| u_{scat}^{Galerkin}\left(\mathbf{x}\right) - u_{scat}^{Analytic} \left(\mathbf{x}\right)\right|^2$ mediato su tutti i punti `XY`
- con il colore vado a rappresentare `opts["N"]`

**Osservazioni**
Si vede che usare "troppi gradi di libertà" risulta inutile se l'integrazione numerica è poco precisa perchè quest'ultima genera un errore che prevale su quello precedente.
Si vedo che raffinando l'integrazione numerica l'errore descresce.

**Possibili implementazioni usando l'approccio IGA**
Si tenga a mente che nell'implementazione da me sviluppata ci sono $3$ tipi di *errore* che entrano in gioco:
- approssimazione di $\Gamma$, questo è vero se si usano delle Bspline per rappresentare delle circonferenze, ma per altre figure questa approssimazione potrebbe non esistere perché $\Gamma$ potrebbe essere rappresentato in modo esatto (un'implementazione delle NURBS) permetterebbe di eliminare questa fonte di errore
- discretizzazione dello spazio delle soluzione (metodo di Galerkin): questo fa parte del metodo BEM e non è possibile eliminarlo
- calcolo numerico degli integrali, anche questo non è eliminabile

**Una nota su una possibile implementazione**
Al momento sono in grado di approssimare $\Gamma$ solo tramite un poligono regolare con polinomi di grado `P=1`, esistono però algoritmi di *degree elevation* che permettono di innalzare il grado polinomiale della curva senza modificarne "il risultato". Una volta implementato questo algoritmo, non complicato ma comunque non banale, sarebbe possibile *dividere l'errore* di approssimazione del dominio dall'errore di discretizzazione dello spazio delle funzioni: potrei avere la stessa identica approssimazione di $\Gamma$ ma uno spazio delle soluzioni diverso. 

**Integrazione numerica** 
Il calcolo numerico degli integrali al momento sfrutta una tecnica molto semplice e basilare:
- ottenuto il dominio di una funzione di base tramite la funzione `basis_range` genero `opts["N"]` punti uniformemente distribuiti nel dominio della funzione (sfrutto la funzione `linspace` di `numpy`)
- la funzione `basis_range` calcola in automatico *l'area* del dominio della funzione di base
- valuto l'integrale $I$ come $I = \frac{A}{M}\sum_{i=1}^{M}f\left(x_i\right)$ dove ho indicato `opts["N"]`  con $M$, `A` è l'area del dominio della funzione, `f` è la funzione da integrare, $x_i$ sono i punti uniformemente distribuiti
- tramite la funzione `adjacency_matrix` (che restituisce una matrice $m_{ij}$ contenente $0,1$) so se il dominio di una funzione di base $i$ si interseca con quello della funzione di base $j$: nel caso le due funzioni si intersechino allora automento il numero di punti  usati per calcolare l'integrale a `opts["N"]`$^2$ per sopprerire al problema della singolarità dell'integrale 
- nel caso esistano dei punti $x_i$ per i quali $f\left(x_i\right)$ assuma "valori problematici", cioè restituisca dei valori `nan`, non considero questi nel calcolo di $I$ e ridefinisco $M$: questo in realtà è fatto in automatico dalla funzioni `nanmean` di `numpy`

E' possibile, passando opportunamente alcuni parametri alla funzione `BEM` generare dei punti *casuali uniformemente distribuiti*, ma ho abbandonato tale approccio molto presto perché non conveniente per integrali $1D$ e $2D$.

In [ ]:
#vettor d'onda
k_in = 30/4*np.asarray([np.sqrt(3.)/2.,0.5])
alpha = np.angle(np.complex(k_in[0],k_in[1]))
wavevector = 7.5#np.sqrt(np.sum(np.power(k_in,2.0)))
print("w:",wavevector)
print("h:",np.pi/(5*wavevector))
I = np.complex(0,1)

xmin = -3
xmax = 3
ymin = -3
ymax = 3

## Preparation

In [ ]:
#function
x0 = 0.0
y0 = 0.0
a = 1.0
b = 1.0
radius = 1.0
def func(t):
    #print(cpz)
    cpx = a*np.cos(2*np.pi*t)+x0#np.random.rand(N)
    cpy = b*np.sin(2*np.pi*t)+y0#np.random.rand(N)
    out = np.zeros(shape=(len(t),2))
    for i in range(len(t)):
        out[i,0] = cpx[i]
        out[i,1] = cpy[i]
    return out

In [ ]:
#punti XY
Nx= 40
Ny = 40
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))
print(len(XY0))

In [ ]:
#plane wave
def plane_wave(xx): # soluzione
    xx = np.asarray(xx)
    theta = np.dot(xx,k_in)
    return np.exp(I*theta)

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,2)
#sh.show()

#defiisco i knot vector
P=1 #polinomial degree
    
xminBs = 0.0
xmaxBs = 1.0
    
#N_arr = [10,20,30,40,50,60,70,80,90,100]
#N_arr = np.arange(11,20)
N_arr = list(np.arange(10,21))+list(np.arange(30,110,10))
#N_arr = [10]

<a id="convergence-cycle"></a>
## Cycle

In [ ]:
#ciclo
j = 0
n = 4
for N in N_arr:    
    
    count = str(j)+"/"+str(len(N_arr))
    
    print(count," : preparazione")
    
    #
    file_dir = "files/BEM/circle-convergence/n="+str(n)+"/"
    suffix = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+"-n="+str(n)
    
    
    kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)
    #kv.show()

    #alloco la Bspline
    bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})
    
    t = np.linspace(0,1,N+1,endpoint=True)#[0:-2]
    cp = func(t)
    for i in range(len(t)):
        bs.set_cp(i,cp[i])
        
    #
    internal = bs.internal_points(XY=XY0,NN=1000,xmin=xminBs,xmax=xmaxBs,opts=None)
    XY = XY0[ np.logical_not(internal) ]
    
    #bs.load("sm-BEM"     ,file_dir+"stiffness_matrix-"            +suffix+".csv")    
    #bs.load("slp-BEM"    ,file_dir+"single_layer_potential-"      +suffix+".csv")    
    #bs.load("sol-BEM"    ,file_dir+"solution-plane_wave-"         +suffix+".csv")
    #bs.load("ind_sol-BEM",file_dir+"indirect_solution-plane_wave-"+suffix+".csv")
    
    #
    print(count," : BEM")
    opts={"print":True,"N":[n]} 
    sol,Xnp,Valnp = bs.BEM(uinc=plane_wave,k=wavevector,XY=XY,opts=opts)

    print(count," : saving")
    bs.save("sm-BEM"     ,file_dir+"stiffness_matrix-"            +suffix+".csv")    
    bs.save("slp-BEM"    ,file_dir+"single_layer_potential-"      +suffix+".csv")    
    bs.save("sol-BEM"    ,file_dir+"solution-plane_wave-"         +suffix+".csv")
    #bs.save("lv-BEM"     ,file_dir+"load_vector-plane_wave-"      +suffix+".csv")
    bs.save("ind_sol-BEM",file_dir+"indirect_solution-plane_wave-"+suffix+".csv")
        
    Uinc = plane_wave(XY)
    total = Uinc + Valnp
    
    index = [tuple(i) for i in XY]
    grafico = pd.DataFrame(index=index,columns=["xy","inc","scat","tot"],dtype=object)
    grafico["xy"] = [ i for i in XY]
    grafico["inc"] = Uinc 
    grafico["scat"] = Valnp
    grafico["tot"] = total

    grafico.to_csv(file_dir+"grafico-plane_wave-"+suffix+".csv",index_label="index")
    
    #grafico
    print(count," : grafico")
    fig = plt.figure ( 0 , figsize = ( 15 , 10 ) )  
    
    cmap = 'RdYlBu'
    
    NN = 1000
    T = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
    xy   = bs.evaluate(T)
    df = pd.DataFrame(xy)
    df = df.rename(columns={0:"x",1:"y"})
   
    plot_sol(fig,331,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
    plot_sol(fig,334,df["x"], df["y"],XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)
    plot_sol(fig,337,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs",cmap)

    plot_sol(fig,332,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.real,"$u_{scat}$ : real",cmap)
    plot_sol(fig,335,df["x"], df["y"],XY[:,0],XY[:,1],Valnp.imag,"$u_{scat}$ : imag",cmap)
    plot_sol(fig,338,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(Valnp),"$u_{scat}$ : abs",cmap)

    plot_sol(fig,333,df["x"], df["y"],XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real",cmap)
    plot_sol(fig,336,df["x"], df["y"],XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag",cmap)
    plot_sol(fig,339,df["x"], df["y"],XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs",cmap)

    plt.tight_layout()
    sol_png = file_dir+"solution-"+suffix+".png"
    plt.savefig(sol_png)
    
    j += 1

    plt.show()


<a id="convergence-norm"></a>
## Norm

In [ ]:
#function
x0 = 0.0
y0 = 0.0
a = 1.0
b = 1.0
radius = 1.0
def func(t):
    #print(cpz)
    cpx = a*np.cos(2*np.pi*t)+x0#np.random.rand(N)
    cpy = b*np.sin(2*np.pi*t)+y0#np.random.rand(N)
    out = np.zeros(shape=(len(t),2))
    for i in range(len(t)):
        out[i,0] = cpx[i]
        out[i,1] = cpy[i]
    return out

In [ ]:
#LN
LN = pd.DataFrame(data=Lebesgue_norm,index=N_arr_2,columns=n_arr)
LN.to_csv("files/BEM/circle-convergence/lebesgue_norm.csv",index_label="index")
LN.head()

In [ ]:
#LNv
LNv = pd.DataFrame(data=Lebesgue_norm_var,index=N_arr_2,columns=n_arr)
LNv.to_csv("files/BEM/circle-convergence/lebesgue_norm_var.csv",index_label="index")
LNv.head()

In [ ]:
#calcolo la norma
def get_float(txt):
    return re.findall("[^a-zA-Z:]([-+]?\d+[\.]?\d*)", txt)

#
#i = 0
#n = 4
N_arr_2 = list(np.arange(10,21))+list(np.arange(30,110,10))
n_arr = [4]
Lebesgue_norm = np.zeros((len(N_arr_2),len(n_arr)))
Lebesgue_norm_var = np.zeros((len(N_arr_2),len(n_arr)))

for k in range(len(n_arr)):
    for i in range(0,len(N_arr_2)):    

        N = N_arr_2[i]
        n = n_arr[k]
        count = str(k)+"/"+str(i)+"       "

        print(count,end="\r")

        #
        file_dir = "files/BEM/circle-convergence/n="+str(n)+"/"
        suffix = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+"-n="+str(n)

        df = pd.read_csv(file_dir+"grafico-plane_wave-"+suffix+".csv")
        df.set_index("index",drop=True,inplace=True)
        df["inc"]  = [np.complex(j) for j in df["inc"]]
        df["scat"] = [np.complex(j) for j in df["scat"]]
        df["tot"]  = [np.complex(j) for j in df["tot"]]

        #
        kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)

        #alloco la Bspline
        bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})

        t = np.linspace(0,1,N+1,endpoint=True)#[0:-2]
        cp = func(t)
        for j in range(len(t)):
            bs.set_cp(j,cp[j])

        #
        T = np.linspace(xminBs,xmaxBs,1000,endpoint=True)
        xy   = bs.evaluate(T)
        uinc = plane_wave(xy)

        #
        XY = np.asarray([ get_float(j) for j in df["xy"] ]).astype(float)

        analytic_pd,analytic = esFFT.analytic_solution_circle(uinc,XY,wmin=-20,wmax=20,radius=radius,\
                                                              wavevector=wavevector,alpha=alpha,\
                                                              opts={"return":"both"})
        
        analytic_pd.to_csv(file_dir+"analytic-solution-plane_wave-"+suffix+".csv",index_label=index)

        df["diff"] = df["scat"] - analytic

        #print(i)
        #Lebesgue_norm[i,k] = np.mean(np.absolute(df["diff"])**2)
        #Lebesgue_norm_var[i,k] = np.var(np.absolute(df["diff"])**2)
        
        LN.at[i,k] = np.mean(np.absolute(df["diff"])**2)
        LNv.at[i,k] = np.var(np.absolute(df["diff"])**2)
        
print("Finished")

In [ ]:
#LN = pd.DataFrame(data=Lebesgue_norm,index=N_arr_2,columns=n_arr)
LN.to_csv("files/BEM/circle-convergence/lebesgue_norm.csv",index_label="index")
LN.head()

In [ ]:
#LNv = pd.DataFrame(data=Lebesgue_norm_var,index=N_arr_2,columns=n_arr)
LNv.to_csv("files/BEM/circle-convergence/lebesgue_norm_var.csv",index_label="index")
LNv.head()

<a id="convergence-grafico"></a>
## Grafico

In [ ]:
#reading
LN = pd.read_csv("files/BEM/circle-convergence/lebesgue_norm.csv")
LN.set_index("index",drop=True,inplace=True)

LNv = pd.read_csv("files/BEM/circle-convergence/lebesgue_norm_var.csv")
LNv.set_index("index",drop=True,inplace=True)

In [ ]:
#grafico
fig = plt.figure(figsize=(15,5))

#
ax = fig.add_subplot(111)
ax.errorbar(LN.index,LN["4"],yerr=LNv["4"],color="blue",label="n=4",linestyle="--",marker=".")
ax.errorbar(LN.index,LN["6"],yerr=LNv["6"],color="green",label="n=6",linestyle="--",marker=".")
ax.errorbar(LN.index,LN["8"],yerr=LNv["8"],color="red",label="n=8",linestyle="--",marker=".")
#ax.scatter(N_arr_2,Lebesgue_norm,color="red",label="calculations")
#plt.xlim(8,41)
plt.grid(True)
plt.xlabel("d.o.f.")
plt.ylabel("Lebesgue norm $L^2$")
plt.legend()
plt.title("Convergence")

plt.show()

# Two circles

In questa sezione ho implementato il BEM anche per $\Gamma$ formato da curve/poligoni disconnessi: ho aggiunto delle funzioni alla libreria `pyBspline` ma esterne alla classe `Bspline` che prendessero in input un array di oggetti `Bspline` e implementassero il BEM sfruttando quanto più possibile quanto già fatto in precedenza.

Il codice scritto funziona con un numero arbitrario di componenti disconnesse, nel notebook ho rappresentato il caso più semplice di due circonferenze dello stesso raggio in modo tale da poter riutilizzare quanto già fatto nella sezione [Circle](#Circle): input della funzione `BEM_disconnected` è infatti l'array `bs_arr = [Left,Right]`, dove `Left` e `Right` sono le Bspline che rappresentano le due circonferenze.

L'unica vera differenza dell'avere $2$ circonferenze anzoché una sola consiste nel calcolo della matrice $A^{Gal}_{ij}$, che per semplicità ho diviso a blocchi (letteralmente anche nel codice, sfruttando i `multindex` dei dataframe di `pandas`), cioè:
\begin{equation*}
A^{Gal}_{ij} = 
\left(\begin{array}{@{}c|c@{}}
A^{LL} &  A^{LR}\\\hline
A^{RL} &  A^{RR}
\end{array}\right)
\end{equation*}
dove $A^{LL}$ è la matrice di stiffness della prima circonferenza, calcolata tramite `Left.stiffness_matrix_BEM` (cioè come fatto in tutte le sezioni precedenti),  $A^{RR}$ è calcolata con `Right.stiffness_matrix_BEM`, mentre le uniche parti il cui calcolo è stato necessario implementare sono le parti fuori diagonali.
In realtà l'implementazione, presente nelle funzioni `stiffness_matrix_BEM_disconnected` e `stiffness_matrix_BEM_disconnected_private`, è stata abbastanza semplice poiché ho riscritto *mutatis mutandis* la funzione `stiffness_matrix_BEM` della classe `Bspline` ma ignorando il discorso sulla raffinazione dell'integrazione perché in questo caso nessun grado di libertà $i$ ha il dominio che si interseca con quello di $j$.

Il calcolo dei blocchi fuori diagonale risulta più veloce rispetto a quelli lungo la diagonale se si guarda il tempo medio di un singolo integrale (perché integro usando sempre `opts["N"]` punti e mai con `opts["N"]`$^2$), ma questo non è vero per l'intero blocco perché $A^{LL}$ è simmetrica, richiede dunque in calcolo di $\frac{n\left(n+1\right)}{2}$ integrali, mentre il calcolo di $A^{LR}$ richiede il calcolo di $n^2$ integrali.

Si ricordi che $A^{Gal}_{ij}$ è simmetrica dunque risulta che $A^{RL}$ è la trasposta (non coniugata) di $A^{LR}$. La matrice $A^{Gal}_{ij}$ viene poi salvata nella variabile `_sm_BEM_disc` e salvata su file.

Sono state poi implemenate le funzioni `single_layer_potential_basis_BEM_disconnected` e `load_vector_BEM_disconnected` che vanno semplicemente a richiamare per ogni Bspline contenuta in `bs_arr` i metodi `single_layer_potential_basis_BEM` e `load_vector_BEM`.

La differenza sta dunque nella soluzione del motodo di Galerkin che "risente dell'interazione" tra le due circonferenze dovuta alla presenza dei termini fuori diagonale (termini che giustamente tendono a zero se le due circonferenza si allontanano), generando una $\psi_N$ che è diversa da $\left[\psi^L_{N/2},\psi^R_{N/2}\right]$, cioè è diversa dal prodotto cartesiano della soluzione di due problemi di scattering indipendenti.

La soluzione del EDP si trova dunque sommando la soluzione dei vari EDP per le due circonferenze: calcolo $u_{scat}^{L}$ generato dalla prima circonferenza, $u_{scat}^{R}$ generato dalla seconda circonferenza e poi li sommo.
Nella variabile `SLP` sono contenute tutte queste informazioni, che ho stampato in parte a schermo tramite `SLP.head()`:
- nella colonna `x` sono presenti tutti i punti di $\mathbf{R}^2$ in cui volgio valutare la soluzione numerica
- nella colonna `value` è presente la soluzione numerica
- nella altre colonne `0,1` (che sono i nomi che ho dato alle due circonferenze, eventualmente si potrebbe cambiarne il nome in `Left,Right`) sono presenti i contributi alla *representation formula* delle due circonferenze.

Alla fine di questa sezione vado a graficare come al solito $u^{inc},u^{scat}$ e $u^{tot}$.

**Una nota su un'ottimizzazione eseguita nel codice**
Si noti che le Bspline `Left` e `Right` non sono state subito allocate nel codice, ma ho prima allocato `bs` e ho calcolato come al solito $A^{Gal}_{ij}$ e i coefficienti $M_j\left(\mathbf{x}\right)$. 

Nell'esempio che qui ho riportato ho usato per semplicità e per ottimizzare i tempi due circonferenze identiche: `Left` e `Right` sono infatti la copia di `bs`.

```
# due cerchi
Left = bs.copy()
Left.traslate_cp(rL)
Left.traslate_slpB(rL)

Right = bs.copy()
Right.traslate_cp(rR)
Right.traslate_slpB(rR)
```

Affinchè vadano a rappresentare due circonferenze distinte ho dovuto traslarle modificando i rispettivi control points: grazie alla partizione dell'unità per traslare la prima circonferenza di un vettore `rL` è sufficiente sommare ad ogni control points `rL`, analogamente per la seconda circonferenza, che è stata traslata di `rR`. Questo viene eseguito dalla funzione `traslate_cp`

Traslando una Bspline la sua stiffness matrix rimane invariata, infatti il codice da me implementato è in grado di *controllare* se `Left` e `Right` avevano già calcolato tale variabile: nella funzione `BEM_disconnected` i blocchi sulla diagonale di  $A^{Gal}_{ij}$ non vengono infatti calcolati perché erano già stati calcolati per `bs`. 

Bisogna però tener conto che una volta traslata una Bspline, i coefficienti $M_j\left(\mathbf{x}\right)$ vanno modificati: anziché ricalcolarli da zero per `Left` e `Right` vado soltanto a *cambiare il nome agli indici*, cioè vado a traslare di `rL` e `rR` i punti $\mathbf{x}$ che compaiono in $M_j\left(\mathbf{x}\right)$. Questo viene eseguito dalla funzione `traslate_slpB`.

Ovviamente i punti in cui voglio calcolare la soluzione numerica dovranno essere contenuti sia nei coefficienti $\mathbf{x}$  di `Left` che di `Right`: all'interno della funzione `BEM_disconnected` si vanno a calcolare soltanto i punti mancanti, questo comporta un notevole risparmio di tempo se i punti richiesti sono tanti e `rL` e `rR` sono *piccoli*.

Si faccia attenzione che per ottimizzare al meglio questo processo ho impostato `rL` e `rL` in modo tale che i nuovi punti traslati si andassero a sovrapporre a quelli vecchi, in modo tale da poter *riciclare* il maggior numero di punti possibili: non ho impostato `rL` e `rL` *a caso* ma li ho impostati come *n* volte la spaziatura dei punti che avevo generato. Questo è comprensibile se visualizzato nel codice
```
#traslo
deltaX = int(1./(x[1]-x[0]))*(x[1]-x[0])
deltaY = int(1./(y[1]-y[0]))*(y[1]-y[0])

rL = [-deltaX,deltaY]
rR = [ deltaX, -deltaY]
```
dove `x` e `y` erano stati definiti in questo modo
```
#punti XY
Nx= 80
Ny = 80
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)
```

## Definition

In [ ]:
#definisco la dimensionaità:
sh = Bs.shape(1,2)
#sh.show()

#defiisco i knot vector
P=1 #polinomial degree
N=20 #base caridnality
xminBs = 0.0
xmaxBs = 1.0


#
#kv = Bs.uniform_open_kv(xmin,xmax,p=P,n=N)#Bs.knot_vector(P,N,v)
#kv = periodic_kv(xmin,xmax,p=P,n=N)
kv = Bs.periodic_kv(xminBs,xmaxBs,p=P,n=N)
#kv.show()

#alloco la Bspline
bs = Bs.Bspline(sh,[kv],properties={"periodic":[True]})

In [ ]:
#
k_in = 30/4*np.asarray([np.sqrt(3.)/2.,0.5])
wavevector = 7.5#np.sqrt(np.sum(np.power(k_in,2.0)))
print("w:",wavevector)
print("h:",np.pi/(5*wavevector))
I = np.complex(0,1)

xmin = -3
xmax = 3
ymin = -3
ymax = 3

In [ ]:
#function
x0 = 0.0
y0 = 0.0
a = 1.0
b = 1.0
radius = 1.0
def func(t):
    #print(cpz)
    cpx = a*np.cos(2*np.pi*t)+x0#np.random.rand(N)
    cpy = b*np.sin(2*np.pi*t)+y0#np.random.rand(N)
    out = np.zeros(shape=(len(t),2))
    for i in range(len(t)):
        out[i,0] = cpx[i]
        out[i,1] = cpy[i]
    return out

In [ ]:
#ATTENZIONE: mi servono dei punti distribuiti in modo uniforme
# per come ho costruito func so che
# func(0.) = func(1.)
# quindi genero un punto in più
t = np.linspace(0,1,N+1,endpoint=True)#[0:-2]
cp = func(t)
for i in range(len(t)):
    #bs._cp[i] = cp[i]
    bs.set_cp(i,cp[i])
cpx = cp[:,0]
cpy = cp[:,1]

In [ ]:
#bs.control_points()

In [ ]:
#files
file_dir = "files/BEM/circle/"
suffix = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)#+".csv"
suffix_png = "P="+str(P)+"-N="+str(N)+"-k="+str(wavevector)+".png"

directory = "files/BEM/two-circles/"

## Stiffness Matrix

In [ ]:
#stiffness matrix
READ = True
SAVE = False
file = directory+"bs-sm-BEM-"+suffix+".csv"
print(file)

if os.path.exists(file) and READ == True :
    sm = bs.load("sm-BEM",file)
else :
    sm = bs.stiffness_matrix_BEM(k=wavevector,opts={"print":True,"N":[6],"ready_sm_BEM":False})
    if SAVE == True :
        bs.save("sm-BEM",file)
sm.head()

In [ ]:
#grafico
sm = bs.stiffness_matrix_BEM()
plot_matrix(sm)

## Single Layer Potential Basis

In [ ]:
#punti XY
Nx= 80
Ny = 80
x = np.linspace(xmin,xmax,Nx)
y = np.linspace(ymin,ymax,Ny)
X,Y = np.meshgrid(x,y)

XY0 = np.zeros((Nx*Ny,2))
XY0[:,0] = X.reshape((Nx*Ny,))
XY0[:,1] = Y.reshape((Nx*Ny,))

internal = bs.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XY0[ np.logical_not(internal) ]
print(len(XY))#," = ",len(XYslp)/3600,"h")

#tolgo elementi interni
#radius = np.asarray([np.sqrt(np.sum(np.power(i,2.0))) for i in XY])
#XYslp = XY#[radius > 1.0]
#print(len(XYslp)," = ",len(XYslp)*3/3600,"h")

In [ ]:
#Single Layer Potential per le funzioni di base
READ = True
SAVE = True

file = directory+"bs-slpB-BEM-"+suffix+".csv"
print(file)

if os.path.exists(file) and READ == True :
    slp = bs.load("slp-BEM",file)
#else :

# I can update it adding some  points
slpB = bs.single_layer_potential_basis_BEM(XY=XY,k=wavevector,opts={"print":False,"N":[6]})

if SAVE == True :
    bs.save("slp-BEM",file)
    
slp.head()

In [ ]:
#traslo
deltaX = int(1./(x[1]-x[0]))*(x[1]-x[0])
deltaY = int(1./(y[1]-y[0]))*(y[1]-y[0])

rL = [-deltaX,deltaY]
rR = [ deltaX, -deltaY]

In [ ]:
#valutazione della Bspline
NN = 1000
t = np.linspace(xminBs,xmaxBs,NN,endpoint=True)
df = pd.DataFrame(index=np.arange(0,len(t)),columns=np.arange(0,6),dtype=object)
index = [  ("left","x") , ("left","y"),\
          ("right","x") , ("right","y")]
mi = pd.MultiIndex.from_tuples(index)
df = df.reindex(columns=mi)

df["t"]      = t

xy = bs.evaluate(t)
#c = center.evaluate(t)
#d =  down.evaluate(t)

df[("left","x")]  = xy[:,0] + rL[0]
df[("left","y")]  = xy[:,1] + rL[1]

df[("right","x")] = xy[:,0] + rR[0]
df[("right","y")] = xy[:,1] + rR[1]


df

In [ ]:
# due cerchi
Left = bs.copy()
Left.traslate_cp(rL)
Left.traslate_slpB(rL)

Right = bs.copy()
Right.traslate_cp(rR)
Right.traslate_slpB(rR)

In [ ]:
# punti interni
XYleft = np.asarray([ np.asarray(i) for i in Left._slp_BEM.index ])
XYright = np.asarray([ np.asarray(i) for i in Right._slp_BEM.index ])

internal = Left.internal_points(XY=XY0,NN=1000,xmin=0.,xmax=1.,opts=None)
XYL = XY0[ np.logical_not(internal) ]

internal = Right.internal_points(XY=XYL,NN=1000,xmin=0.,xmax=1.,opts=None)
XY = XYL[ np.logical_not(internal)] 
print(len(XY))

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 15, 5 ) )
#
ax = fig.add_subplot(111)
plt.plot(df[("left","x")], df[("left","y")], color= "black",label="up")
plt.plot(df[("right","x")], df[("right","y")], color= "black",label="center")
plt.scatter(XY[:,0],XY[:,1], color= "blue",label="points",s=0.1)

#plt.scatter(XYleft[:,0],XYleft[:,1], color= "red",label="points",s=0.1)
#plt.scatter(XYright[:,0],XYright[:,1], color= "green",label="points",s=0.1)

#plt.grid()
#plt.legend()
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
ax.set_aspect('equal')
plt.show()

In [ ]:
#Single Layer Potential per le funzioni di base
READ = True
SAVE = True

file = directory+"Left-slpB-BEM-"+suffix+".csv"
print(file)

if os.path.exists(file) and READ == True :
    slpBL = Left.load("slp-BEM",file)
#else :

slpBL = Left.single_layer_potential_basis_BEM(XY=XY,k=wavevector,\
                                              opts={"print":False,"prec":1e-6,"ready_slp_BEM":True})
if SAVE == True :
    Left.save("slp-BEM",directory+"Left-slpB-BEM-"+suffix+".csv")
    
slpBL.head()

In [ ]:
#Single Layer Potential per le funzioni di base
READ = True
SAVE = True

file = directory+"Right-slpB-BEM-"+suffix+".csv"
print(file)

if os.path.exists(file) and READ == True :
    slpBR = Right.load("slp-BEM",file)
#else :

slpBR = Right.single_layer_potential_basis_BEM(XY=XY,k=wavevector,\
                                              opts={"print":False,"prec":1e-6,"ready_slp_BEM":True})
if SAVE == True :
    Right.save("slp-BEM",directory+"Right-slpB-BEM-"+suffix+".csv")
    
slpBR.head()

## Solution

In [ ]:
#plane wave
def plane_wave(xx): # soluzione
    xx = np.asarray(xx)
    theta = np.dot(xx,k_in)
    return np.exp(1.j*theta)

In [ ]:
# stiffness matrix
SM = pd.read_csv(directory+"SM-"+suffix+".csv",skiprows=2)
del SM["index"]
SM = SM.applymap(np.complex)
index,mi,last,first = Bs.multi_index_disconnected([Left,Right],[0,1],None)
df0 = pd.DataFrame(data = np.asarray(SM),columns=index,index=index)
SM = df0.reindex(columns=mi,index=mi)
Bs._sm_BEM_disc = SM.copy()
Bs._ready_sm_BEM_disc = True
SM.head()

In [ ]:
#metodo di Galerkin
bs_arr = [Left,Right]
opts = {"print":False,"N":[6]}
SLP,Xnp,Valnp = Bs.BEM_disconnected(bs=bs_arr,uinc=plane_wave,k=wavevector,XY=XY,opts=opts)

In [ ]:
SLP.head()

In [ ]:
SLP.to_csv(directory+"solution-"+suffix+".csv",index_label="index")
Bs._sm_BEM_disc.to_csv(directory+"SM-"+suffix+".csv",index_label="index")
Bs._lv_BEM_disc.to_csv(directory+"LV-"+suffix+".csv",index_label="index")

In [ ]:
plot_matrix(Bs.get_block(SM,0,0,drop=True))

In [ ]:
plot_matrix(Bs.get_block(SM,1,1,drop=True))

In [ ]:
plot_matrix(Bs.get_block(SM,0,1,drop=True))

In [ ]:
plot_matrix(Bs.get_block(SM,1,0,drop=True))

In [ ]:
#uinc
Uinc = plane_wave(XY)#.reshape(Nx,Ny).transpose()
scat = np.asarray(SLP["value"],dtype=np.complex)#Valnp
total = Uinc + scat

In [ ]:
#grafico
fig = plt.figure ( 0 , figsize = ( 13 , 13 ) )

cmap = 'RdYlBu'

x = [df[("left","x")],df[("right","x")]]
y = [df[("left","y")],df[("right","y")]]
   
plot_sol_disc(fig,331,x,y,XY[:,0],XY[:,1],Uinc.real,"$u_{inc}$ : real",cmap)
plot_sol_disc(fig,334,x,y,XY[:,0],XY[:,1],Uinc.imag,"$u_{inc}$ : imag",cmap)
plot_sol_disc(fig,337,x,y,XY[:,0],XY[:,1],np.absolute(Uinc),"$u_{inc}$ : abs",cmap)

plot_sol_disc(fig,332,x,y,XY[:,0],XY[:,1],scat.real,"$u_{scat}$ : real",cmap)
plot_sol_disc(fig,335,x,y,XY[:,0],XY[:,1],scat.imag,"$u_{scat}$ : imag",cmap)
plot_sol_disc(fig,338,x,y,XY[:,0],XY[:,1],np.absolute(scat),"$u_{scat}$ : abs",cmap)

plot_sol_disc(fig,333,x,y,XY[:,0],XY[:,1],total.real,"$u_{tot}$ : real",cmap)
plot_sol_disc(fig,336,x,y,XY[:,0],XY[:,1],total.imag,"$u_{tot}$ : imag",cmap)
plot_sol_disc(fig,339,x,y,XY[:,0],XY[:,1],np.absolute(total),"$u_{tot}$ : abs",cmap)

plt.tight_layout()

sol_png = directory+"solution-2-"+suffix_png
plt.savefig(sol_png)

plt.show()